## Notebook principal
<br>
Implémentation du modèle (fonctions objectifs, contraintes et résolution/optimisation).

### Importations

In [1]:
import gurobipy as gp # solver
from gurobipy import GRB
import numpy as np

In [2]:
from InstanceClass import Instance # custom class built to load and encode .json files (instances)

### Loading the data

In [3]:
try:
    instance = Instance(path="toy_instance.json")
    instance.build_instance()
except FileNotFoundError:
    instance = Instance(path="data/toy_instance.json")
    instance.build_instance()

In [4]:
instance.variables # check that everything works as expected

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

### Création de la matrice des affectations
<br>
On définit la matrice A des affections des collaborateurs sur les projets (de taille n_c ∗ h  ∗ n_p ∗ n): 
<br>
$
𝐴 = {𝑎_{𝑖,𝑗,𝑘,l}}
$
<br>


On en extrait une matrice 𝐴𝑝 = {𝑎𝑝𝑗,𝑘} indiquant si à l’instant j le projet vient d’être
rendu 


In [5]:
I=3
J=5
L=5
K=3

In [6]:
def solution_creation(variables):
    shape=(variables["NC"],variables["H"],variables["NP"],variables["NA"])
    return shape

def calcul_work(Affectation):
    rep = [np.sum([Affectation[i][j] for i in range(3)],axis=0) for j in range(np.shape(Affectation)[1])]
    return np.array(rep)

def calcul_temporel_work(work,horizon):
    rep = [[gp.quicksum(work_project[0:(i+1)]) for i in range(horizon)] for work_project in work]
    return np.array(rep)

def calcul_work_final(work):
    rep=gp.quicksum(work)  
    return rep

def calcul_Ap(rep,cost_projet):
    X = [[(rep[i][j] >= cost_projet[i]) for j in range(len(rep))] for i in range(len(cost_projet))]
    return np.array(X)

In [10]:
# données propres au problème que l'on va modéliser
shape = solution_creation(instance.variables)
COST_PROJECT = instance.variables["COST_PROJECT"]
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]
GAIN = instance.variables["GAIN"]

### Instanciation du modèle

In [11]:
model = gp.Model("matrix1") # instanciation du modèle

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


### Définition des variables

In [14]:
# Create variables
Affectation = model.addMVar(shape=(I,J,L,K), vtype=GRB.BINARY, name="Affectation") # matrice personnes/jours/projet/compétence (tous les projets)
Done_Project = model.addMVar(shape=L, vtype=GRB.BINARY, name="Done_Project") # matrice projets/temps (tous les projets)
Ap = calcul_Ap(calcul_work(Affectation),COST_PROJECT) # pas utilisé pour le moment
final_work = calcul_work_final(calcul_work(Affectation)) # somme des travaux menés par projet (sert à vérifier qu'un projet est complété)
# dimension de final_work : matrice projets/compétences (somme sur les employés par compétence et par projet, au temps final)
temp_work = calcul_temporel_work(calcul_work(Affectation),instance.variables["H"]) # somme cumulée dans le temps du travail effectué par projet et 
# par compétence (on veut l'utiliser plutôt que final_work à terme) ; temp_work[-1] = finl_work

### Définition des fonctions objectifs
<br>
Note : il manque les fonctions objectifs 2 et 3 ainsi que le multi-objectifs

In [18]:
model.setObjective(Done_Project @ np.transpose(GAIN), GRB.MAXIMIZE) # partie 1 (avant le -) de la FO 1

a[0, 0, 0, 0] : vaut 1 si l'employé 0 a travaillé au temps 0 sur le projet 0 selon la compétence 0
a[0, 0, 0, i] sum(i) : vaut 1 si l'employé 0 a travaillé au temps 0 sur le projet 0
a[0, 0, j, i] sum(i, j) : vaut 1 si l'employé 0 a travaillé au temps 0 sur un projet quelconque
a[0, ?, ?, ?] : employé 0 et je veux savoir sur combien de projet il a travaillé

In [30]:
Q1 = gp.quicksum(Affectation[0])
Q2 = [gp.quicksum(q) for q in Q1]
Q2

[<MLinExpr ()>
 array( Affectation[0,0,0,0] + Affectation[0,1,0,0] + Affectation[0,2,0,0] + Affectation[0,3,0,0] + Affectation[0,4,0,0] + Affectation[0,0,0,1] + Affectation[0,1,0,1] + Affectation[0,2,0,1] + Affectation[0,3,0,1] + Affectation[0,4,0,1] + Affectation[0,0,0,2] + Affectation[0,1,0,2] + Affectation[0,2,0,2] + Affectation[0,3,0,2] + Affectation[0,4,0,2]),
 <MLinExpr ()>
 array( Affectation[0,0,1,0] + Affectation[0,1,1,0] + Affectation[0,2,1,0] + Affectation[0,3,1,0] + Affectation[0,4,1,0] + Affectation[0,0,1,1] + Affectation[0,1,1,1] + Affectation[0,2,1,1] + Affectation[0,3,1,1] + Affectation[0,4,1,1] + Affectation[0,0,1,2] + Affectation[0,1,1,2] + Affectation[0,2,1,2] + Affectation[0,3,1,2] + Affectation[0,4,1,2]),
 <MLinExpr ()>
 array( Affectation[0,0,2,0] + Affectation[0,1,2,0] + Affectation[0,2,2,0] + Affectation[0,3,2,0] + Affectation[0,4,2,0] + Affectation[0,0,2,1] + Affectation[0,1,2,1] + Affectation[0,2,2,1] + Affectation[0,3,2,1] + Affectation[0,4,2,1] + Affectation

In [ ]:
for i in range(staff):
    Q0 = a[i].quicksum(axis=0) # on somme sur le temps
    Q1 = Q0/Q0.quicksum(axis=0)
    N = quicksum(Q1, axis=)
model.setObjective()


In [ ]:
model.setObjective()

### Définition des contraintes
<br>
Note : toutes les contraintes sont faites en v.1

In [19]:
model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1,1,1]))) <= 1 for i in range(I) for j in range(J) for l in range(L)), name="Quantity_worked_Constraint") 
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1,1,1]))) <= 1-CONGES[i][j] for i in range(I) for j in range(J) for l in range(L)), name="Day_off_Constraint") 
model.addConstrs((final_work[i] >= COST_PROJECT[i]*Done_Project[i] for i in range(L)), name="Done") 
model.addConstrs((Done_Project[i] <= 1 for i in range(L)), name="Only_Done_One_Time") 

{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>}

### Optimisation du modèle

In [16]:
shape = solution_creation(instance.variables)
COST_PROJECT = instance.variables["COST_PROJECT"]
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]
GAIN = instance.variables["GAIN"]

model = gp.Model("matrix1")

# Create variables
Affectation = model.addMVar(shape=(I,J,L,K), vtype=GRB.BINARY, name="Affectation")
Done_Project = model.addMVar(shape=L, vtype=GRB.BINARY, name="Done_Project")
Ap = calcul_Ap(calcul_work(Affectation),COST_PROJECT)
final_work = calcul_work_final(calcul_work(Affectation))
temp_work = calcul_temporel_work(calcul_work(Affectation),instance.variables["H"])

model.setObjective(Done_Project @ np.transpose(GAIN), GRB.MAXIMIZE)

model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1,1,1]))) <= 1 for i in range(I) for j in range(J) for l in range(L)), name="Quantity_worked_Constraint") 
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1,1,1]))) <= 1-CONGES[i][j] for i in range(I) for j in range(J) for l in range(L)), name="Day_off_Constraint") 
model.addConstrs((final_work[i] >= COST_PROJECT[i]*Done_Project[i] for i in range(L)), name="Done") 
model.addConstrs((Done_Project[i] <= 1 for i in range(L)), name="Only_Done_One_Time") 
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i3-6006U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 790 rows, 460 columns and 5430 nonzeros
Model fingerprint: 0x40feb962
Variable types: 0 continuous, 460 integer (460 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 372 rows and 135 columns
Presolve time: 0.00s
Presolved: 23 rows, 95 columns, 190 nonzeros
Variable types: 0 continuous, 95 integer (95 binary)

Root relaxation: infeasible, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [8]:
gp.quicksum(Affectation[0][0] @ np.transpose(np.array([1,1,1])))

<MLinExpr ()>
array( Affectation[0,0,0,0] + Affectation[0,0,0,1] + Affectation[0,0,0,2] + Affectation[0,0,1,0] + Affectation[0,0,1,1] + Affectation[0,0,1,2] + Affectation[0,0,2,0] + Affectation[0,0,2,1] + Affectation[0,0,2,2] + Affectation[0,0,3,0] + Affectation[0,0,3,1] + Affectation[0,0,3,2] + Affectation[0,0,4,0] + Affectation[0,0,4,1] + Affectation[0,0,4,2])

In [9]:
Affectation[0][0]

<MVar (5, 3)>
array([[<gurobi.Var Affectation[0,0,0,0] (value -0.0)>,
        <gurobi.Var Affectation[0,0,0,1] (value -0.0)>,
        <gurobi.Var Affectation[0,0,0,2] (value -0.0)>],
       [<gurobi.Var Affectation[0,0,1,0] (value -0.0)>,
        <gurobi.Var Affectation[0,0,1,1] (value -0.0)>,
        <gurobi.Var Affectation[0,0,1,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,2,0] (value -0.0)>,
        <gurobi.Var Affectation[0,0,2,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,2,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,3,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,3,1] (value 1.0)>,
        <gurobi.Var Affectation[0,0,3,2] (value -0.0)>],
       [<gurobi.Var Affectation[0,0,4,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,4,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,4,2] (value -0.0)>]])

In [10]:
i = 2

In [11]:
COST_PROJECT[i]*Done_Project[i]

<MLinExpr (3,)  >
array([ Done_Project[2],  0.0 Done_Project[2],  2.0 Done_Project[2]])

In [12]:
COST_PROJECT[i]*Done_Project[i]

<MLinExpr (3,)  >
array([ Done_Project[2],  0.0 Done_Project[2],  2.0 Done_Project[2]])

In [13]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 395 rows, 230 columns and 2715 nonzeros
Model fingerprint: 0x8e5e7eaf
Variable types: 0 continuous, 230 integer (230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 23 rows, 95 columns, 190 nonzeros

Continuing optimization...


Explored 1 nodes (40 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 70 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+01, best bound 7.000000000000e+01, gap 0.0000%


In [14]:
Affectation[1]

<MVar (5, 5, 3)>
array([[[<gurobi.Var Affectation[1,0,0,0] (value 0.0)>,
         <gurobi.Var Affectation[1,0,0,1] (value 0.0)>,
         <gurobi.Var Affectation[1,0,0,2] (value 0.0)>],
        [<gurobi.Var Affectation[1,0,1,0] (value 0.0)>,
         <gurobi.Var Affectation[1,0,1,1] (value 0.0)>,
         <gurobi.Var Affectation[1,0,1,2] (value 0.0)>],
        [<gurobi.Var Affectation[1,0,2,0] (value 0.0)>,
         <gurobi.Var Affectation[1,0,2,1] (value 0.0)>,
         <gurobi.Var Affectation[1,0,2,2] (value 0.0)>],
        [<gurobi.Var Affectation[1,0,3,0] (value 0.0)>,
         <gurobi.Var Affectation[1,0,3,1] (value 0.0)>,
         <gurobi.Var Affectation[1,0,3,2] (value 0.0)>],
        [<gurobi.Var Affectation[1,0,4,0] (value 0.0)>,
         <gurobi.Var Affectation[1,0,4,1] (value 0.0)>,
         <gurobi.Var Affectation[1,0,4,2] (value 0.0)>]],

       [[<gurobi.Var Affectation[1,1,0,0] (value -0.0)>,
         <gurobi.Var Affectation[1,1,0,1] (value 1.0)>,
         <gurobi.Var Af

In [15]:
for v in model.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % model.ObjVal)

Affectation[0,0,0,0] -0
Affectation[0,0,0,1] -0
Affectation[0,0,0,2] -0
Affectation[0,0,1,0] -0
Affectation[0,0,1,1] -0
Affectation[0,0,1,2] 0
Affectation[0,0,2,0] -0
Affectation[0,0,2,1] 0
Affectation[0,0,2,2] 0
Affectation[0,0,3,0] 0
Affectation[0,0,3,1] 1
Affectation[0,0,3,2] -0
Affectation[0,0,4,0] 0
Affectation[0,0,4,1] 0
Affectation[0,0,4,2] -0
Affectation[0,1,0,0] -0
Affectation[0,1,0,1] -0
Affectation[0,1,0,2] -0
Affectation[0,1,1,0] -0
Affectation[0,1,1,1] -0
Affectation[0,1,1,2] 0
Affectation[0,1,2,0] -0
Affectation[0,1,2,1] 0
Affectation[0,1,2,2] 0
Affectation[0,1,3,0] 0
Affectation[0,1,3,1] 1
Affectation[0,1,3,2] -0
Affectation[0,1,4,0] 0
Affectation[0,1,4,1] 0
Affectation[0,1,4,2] -0
Affectation[0,2,0,0] -0
Affectation[0,2,0,1] -0
Affectation[0,2,0,2] -0
Affectation[0,2,1,0] -0
Affectation[0,2,1,1] 1
Affectation[0,2,1,2] 0
Affectation[0,2,2,0] -0
Affectation[0,2,2,1] 0
Affectation[0,2,2,2] -0
Affectation[0,2,3,0] 0
Affectation[0,2,3,1] 0
Affectation[0,2,3,2] -0
Affectation